### Analysis of Recall 

In [1]:
import pandas as pd
import glob

In [2]:
memory_df = pd.read_csv('../data/01_interview_data/recall_vr_driving_e.csv')

subj_restrict_counter = 39
memory_df = memory_df.loc[:subj_restrict_counter]
print(len(memory_df))
memory_df.head()

40


,subject,binge_drinking,buckle_up,diabetes,HIV,text_driving,drugged_driving,smoking,sun_protection,wash_hands,...,marijuana,tech_addiction,healthy_diet,brunch,burger,coffee,education_donation,furniture,hotel,condition
0,sub001,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,sub002,1,1,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
2,sub003,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
3,sub004,0,0,0,0,0,1,0,0,0,...,1,0,1,1,0,1,0,0,1,1
4,sub005,0,0,0,0,0,1,0,0,1,...,0,0,0,1,0,0,0,0,0,1


In [3]:
#recognition_df = pd.read_csv('../data/02_qualtrics_data/vr_billboard_qualtrics_cleaned.csv')#

#recognition_df = recognition_df.drop(['date', 'duration_sec', 'age', 'gender',
#       'virtual sickness_1_discomfort', 'virtual_sickness_2_fatigue',
#       'virtual_sickness_3_eyestrain', 'virtual_sickness_4_difficultyfocusing',
#       'virtual_sickness_5_headache', 'virtual_sickness_6_fullnessofhead',
#       'virtual_sickness_7_blurredvision', 'virtual_sickness_8_dizzy',
#       'virtual_sickness_9_vertigo', 'spatial_presence_1_transported',
#       'spatial_presence_2_threedimensional',
#       'spatial_presence_3_realisticproportions',
#       'spatial_presence_4_movefreely', 'spatial_presence_5_physicallypresent',
#       'distractor_moonrise_hotel',
#       'distractor_pollen', 'distractor_exit_32', 'distractor_nhtsa'], axis=1)

#recognition_df.head()

In [4]:
folders = glob.glob("../data/00_gaze_data/sub*")
folders.sort()
subjs = []
for f in folders:
    subjs.append(f[-6:])
print(len(subjs))  

# this needs to be removed -- it is only here to restrict ppts to the ones we have in the preliminary analysis to check if everything workds
subjs = subjs[:subj_restrict_counter+1]
print(len(subjs))  
subjs[:3]

40
40


['sub001', 'sub002', 'sub003']

In [6]:
for i in range(len(subjs)):
    #read this participant's fixation data
    curr_sub_fixation_file = '../data/00_gaze_data/fixation_billboards/'  + subjs[i] +'_fixations_billboards.csv'
    out_file_name = '../data/03_fixation_billboards_recall_recognition/'  + subjs[i] +'_fixations_billboards_recall_recognition_emointegrated.csv'
   
    curr_sub_fix_df = pd.read_csv(curr_sub_fixation_file)
    curr_sub_fix_df['billboard_image_at_position_text_emotion'] = curr_sub_fix_df.loc[:, 'billboard_image_at_position_text']
    curr_sub_fix_df = curr_sub_fix_df.replace(to_replace=r'health_\d*_', value='', regex=True)
    curr_sub_fix_df = curr_sub_fix_df.replace(to_replace=r'commercial_\d*_', value='', regex=True)
    curr_sub_fix_df['billboard_image_at_position_text'] = curr_sub_fix_df['billboard_image_at_position_text'].replace(to_replace=r'_emo', value='', regex=True)
    curr_sub_fix_df['billboard_image_at_position_text'] = curr_sub_fix_df['billboard_image_at_position_text'].replace(to_replace=r'_neu', value='', regex=True)
    curr_sub_fix_df.head()


    #subselect the position, billboard, and fixations for this subject
    position_billboard_fix = curr_sub_fix_df[["position_index", "billboard_image_at_position_text", "billboard_image_at_position_text_emotion", "fix_bill", "count"]]
    print(position_billboard_fix['fix_bill'].values)
    #position_billboard_fix 

    #munge the recall df around
    current_subject_recall = memory_df[memory_df['subject']==subjs[i]]
    current_subject_recall = current_subject_recall.drop('subject', axis= 1)
    current_subject_recall = current_subject_recall.T
    current_subject_recall['billboard'] = current_subject_recall.index
    current_subject_recall.rename( columns={i :'recall'}, inplace=True )
    #current_subject_recall.head()


    #merge fixation and recall df
    position_billboard_fix_recall = position_billboard_fix.merge(current_subject_recall, left_on = 'billboard_image_at_position_text', right_on = 'billboard')
    position_billboard_fix_recall = position_billboard_fix_recall.drop('billboard', axis=1)

    print(position_billboard_fix_recall['fix_bill'].values)

    
    #save to file
    # position_billboard_fix_recall_recognition.to_csv(out_file_name)
    position_billboard_fix_recall.to_csv(out_file_name)
    print('---')
    
    
print('done')

[ 0.  2.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 13.  0.  0.  0. 17.  0.
  0.  0.]
[ 0.  2.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 13.  0.  0.  0. 17.  0.
  0.  0.]
---
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]
---
[ 1.  0.  3.  0.  5.  0.  0.  8.  9.  0. 11.  0.  0.  0.  0.  0. 17.  0.
  0.  0.]
[ 1.  0.  3.  0.  5.  0.  0.  8.  9.  0. 11.  0.  0.  0.  0.  0. 17.  0.
  0.  0.]
---
[ 1.  2.  3.  4.  5.  6.  7.  8.  0. 10.  0. 12. 13. 14. 15. 16. 17. 18.
 19. 20.]
[ 1.  2.  3.  4.  5.  6.  7.  8.  0. 10.  0. 12. 13. 14. 15. 16. 17. 18.
 19. 20.]
---
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]
---
[ 0.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17. 18.
 19. 20.]
[ 0.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17. 18.
 19. 20.]
---
[ 1.  0.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16.